In [13]:
import pandas as pd
import numpy as np
import collections
import json
import pandas as pd
import plotly.express as px
import os
import pickle
import plotly.graph_objects as go

In [2]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
file_name="../dataset/consultation_referendum_bill.json"
df= pd.read_json(file_name, orient="index") 

In [95]:
df["id"].describe()

count    6.806000e+03
mean     5.387992e+08
std      3.098195e+08
min      1.497540e+05
25%      2.665793e+08
50%      5.430482e+08
75%      8.069444e+08
max      1.073541e+09
Name: id, dtype: float64

In [21]:
documents_q1=df["What are your views on the proposed arrangements for managing the referendum?"]
documents_q2=df["What are your views on the proposed technical changes to polling and count arrangements?"]
documents_q3=df["What are your views on the proposed changes to rules on permissible participants?"]


In [22]:
type_indiv=df["Are you responding as an individual or an organisation?"]

In [30]:
type_indiv.describe()

count           6806
unique             2
top       Individual
freq            6804
Name: Are you responding as an individual or an organisation?, dtype: object

In [33]:
type_indiv.unique()

array(['Individual', 'No Answered'], dtype=object)

In [34]:
type_indiv.value_counts()

Individual     6804
No Answered       2
Name: Are you responding as an individual or an organisation?, dtype: int64

In [29]:
fig = px.histogram(type_indiv,
                  title="Type of Individual")
fig.update_layout(showlegend=True)
fig.show()

In [35]:
!pip install gender-guesser

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 379 kB 1.3 MB/s eta 0:00:01


In [36]:
import gender_guesser.detector as gender

In [37]:
d = gender.Detector()

In [38]:
name= df["What is your name?"]
name

0                  Fiona Smith
1                    Anonymous
2                    Anonymous
3                   Craig Lynn
4                Kieran Lennon
                 ...          
6801     Craig Herbert Raeside
6802                 Anonymous
6803    Howard League Scotland
6804                 Anonymous
6805           David Henderson
Name: What is your name?, Length: 6806, dtype: object

In [40]:
print(d.get_gender(name[0].split(" ")[0]), name[0])

female Fiona Smith


In [87]:
gender=df[df["What is your name?"]!="Anonymous"]
gender=gender[gender["What is your name?"]!="N/A"]
gender=gender[gender["What is your name?"]!="n/a"]
gender=gender[gender["What is your name?"]!="None"]
gender=gender[gender["What is your name?"]!="Mr"]
gender=gender[gender["What is your name?"]!="Redacted text"]


In [88]:
name=gender["What is your name?"].apply(lambda x: x.split(" ")[0])
gender["What is your name?"]=name


In [90]:
gender["What is your name?"].value_counts()

John        116
David       106
James        90
Robert       84
William      58
           ... 
NAMBLA        1
Unionist      1
Mac           1
DEBORAH       1
STEWART       1
Name: What is your name?, Length: 813, dtype: int64

In [91]:
gender_guess=gender["What is your name?"].apply(lambda x: d.get_gender(x.split(" ")[0]))
fig = px.histogram(gender_guess,
                  title="gender")
fig.update_layout(showlegend=True)
fig.show()


In [84]:
fig = px.histogram(name,
                  title="name")
fig.update_layout(showlegend=True)
fig.show()

In [63]:
name.describe()

count     2918
unique    2785
top       John
freq         5
Name: What is your name?, dtype: object

In [24]:
model = SentenceTransformer('all-mpnet-base-v2')

In [26]:
q1_embeddings = model.encode(documents_q1, batch_size = 8, show_progress_bar = True)

Batches:   0%|          | 0/851 [00:00<?, ?it/s]

In [27]:
q2_embeddings = model.encode(documents_q2, batch_size = 8, show_progress_bar = True)


Batches:   0%|          | 0/851 [00:00<?, ?it/s]

In [28]:
q3_embeddings = model.encode(documents_q3, batch_size = 8, show_progress_bar = True)

Batches:   0%|          | 0/851 [00:00<?, ?it/s]